In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

2024-07-10 22:01:46.237750: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-10 22:01:46.323439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-10 22:01:46.440350: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-10 22:01:46.443059: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-10 22:01:46.538843: I tensorflow/core/platform/cpu_feature_gua

In [ ]:

# Function to load images from a folder
def load_images_from_folder(folder, augmentation=False):
    images = []
    datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1,
                                 shear_range=0.1, zoom_range=0.1, horizontal_flip=True, fill_mode='nearest') if augmentation else None

    for filename in os.listdir(folder):
        img = load_img(os.path.join(folder, filename), color_mode='grayscale', target_size=(155, 220))
        if img is not None:
            img = img_to_array(img)
            img = np.expand_dims(img, axis=0)
            if augmentation:
                for batch in datagen.flow(img, batch_size=1):
                    images.append(batch[0])
                    break
            else:
                images.append(img[0])

    return np.array(images)


In [ ]:

# Function to load datasets
def load_datasets(base_path, augmentation=False):
    all_images = []
    all_labels = []
    
    for i in range(1, 70):
        if i in [5,7,8,10,11]:
            continue
        real_folder = os.path.join(base_path, f'{i:03}')
        forge_folder = os.path.join(base_path, f'{i:03}_forg')
        
        real_images = load_images_from_folder(real_folder, augmentation)
        forge_images = load_images_from_folder(forge_folder, augmentation)
        
        images = np.concatenate((real_images, forge_images), axis=0)
        labels = np.array([1] * len(real_images) + [0] * len(forge_images))
        
        all_images.append(images)
        all_labels.append(labels)
    
    all_images = np.concatenate(all_images, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    
    return all_images, all_labels

# Function to load datasets
def load_datasets_test(base_path, augmentation=False):
    all_images = []
    all_labels = []
    
    for i in range(49, 70):
        real_folder = os.path.join(base_path, f'{i:03}')
        forge_folder = os.path.join(base_path, f'{i:03}_forg')
        
        real_images = load_images_from_folder(real_folder, augmentation)
        forge_images = load_images_from_folder(forge_folder, augmentation)
        
        images = np.concatenate((real_images, forge_images), axis=0)
        labels = np.array([1] * len(real_images) + [0] * len(forge_images))
        
        all_images.append(images)
        all_labels.append(labels)
    
    all_images = np.concatenate(all_images, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    
    return all_images, all_labels
# Base path to the datasets
train_path = 'sign_data/train'
test_path = 'sign_data/test'


In [ ]:
# Load and normalize datasets
train_images, train_labels = load_datasets(train_path, augmentation=True)
test_images, test_labels = load_datasets_test(test_path, augmentation=False)

train_images = train_images / 255.0
test_images = test_images / 255.0

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Function to build the Signet model with enhancements
def build_signet_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(64, (11, 11), strides=(4, 4), activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    x = Conv2D(128, (5, 5), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    model = Model(inputs, x)
    return model

input_shape = (155, 220, 1)
sig_model = build_signet_model(input_shape)


In [ ]:
# Define a custom callback for live plotting
class PlotLosses(Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        self.fig, self.ax = plt.subplots(1, 2, figsize=(12, 6))

    def on_epoch_end(self, epoch, logs={}):
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('accuracy'))
        self.val_acc.append(logs.get('val_accuracy'))
        self.i += 1

        self.ax[0].clear()
        self.ax[0].plot(self.x, self.losses, label="loss")
        self.ax[0].plot(self.x, self.val_losses, label="val_loss")
        self.ax[0].legend()
        self.ax[0].set_title('Loss')

        self.ax[1].clear()
        self.ax[1].plot(self.x, self.acc, label="accuracy")
        self.ax[1].plot(self.x, self.val_acc, label="val_accuracy")
        self.ax[1].legend()
        self.ax[1].set_title('Accuracy')

        plt.pause(0.1)

In [ ]:
# Custom function for the Lambda layer
def custom_abs_diff(tensors):
    x, y = tensors
    return tf.abs(x - y)

In [ ]:

# Siamese model
def siamese_model(base_model, input_shape):
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)
    processed_a = base_model(input_a)
    processed_b = base_model(input_b)
    L1_layer = Lambda(custom_abs_diff)([processed_a, processed_b])
    prediction = Dense(1, activation='sigmoid')(L1_layer)
    model = Model(inputs=[input_a, input_b], outputs=prediction)
    return model

siamese_net = siamese_model(sig_model, input_shape)
siamese_net.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Function to generate pairs of images for training
def make_pairs(images, labels):
    pair_images = []
    pair_labels = []
    num_classes = len(np.unique(labels))
    idx = [np.where(labels == i)[0] for i in range(num_classes)]
    for idxA in range(len(images)):
        current_image = images[idxA]
        label = labels[idxA]
        # Positive Pair
        idxB = np.random.choice(idx[label])
        pos_image = images[idxB]
        pair_images.append([current_image, pos_image])
        pair_labels.append(1)
        # Negative Pair
        neg_label = np.random.choice(list(set(range(num_classes)) - set([label])))
        idxB = np.random.choice(idx[neg_label])
        neg_image = images[idxB]
        pair_images.append([current_image, neg_image])
        pair_labels.append(0)
    return np.array(pair_images), np.array(pair_labels)

pairs_train, labels_train = make_pairs(X_train, y_train)
pairs_val, y_val = make_pairs(X_val, y_val)
pairs_test, labels_test = make_pairs(test_images, test_labels)

# Enhanced training with callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00003)
plot_losses = PlotLosses()
history = siamese_net.fit(
    [pairs_train[:, 0], pairs_train[:, 1]], labels_train, 
    validation_data=([pairs_val[:, 0], pairs_val[:, 1]], y_val), 
    batch_size=8, epochs=50, 
    callbacks=[early_stopping, reduce_lr]
)


In [ ]:
siamese_net.save('signet_model_augmented_b8.keras')

In [ ]:

# Function to preprocess an image
def preprocess_image(image_path):
    img = load_img(image_path, color_mode='grayscale', target_size=(155, 220))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0
    return img

# Ensure custom Lambda function is registered during model loading
custom_objects = {'custom_abs_diff': custom_abs_diff}

# Load the model with custom objects
model = load_model('signet_model_augmented.keras', custom_objects=custom_objects, compile=False)

# Function to evaluate the genuineness of two signatures
def evaluate_signature(img1_path, img2_path):
    img1 = preprocess_image(img1_path)
    img2 = preprocess_image(img2_path)
    prediction = model.predict([img1, img2])
    return prediction[0][0] * 100


In [ ]:
# Evaluate training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

In [ ]:

# Example usage
fake_img_path = 'img2.jpeg'
real_img_path = 'img4.jpeg'
genuineness = evaluate_signature(fake_img_path, real_img_path)

if genuineness > 75:
    print(f"genuine: {genuineness:.2f}%")
else:
    print(f"fraud: {genuineness:.2f}%")

